The `HttpClient` class exposes a `modern API for HTTP client operations`, ***replacing*** the old `WebClient` and `WebRequest/WebResponse` types

In particular:  
- A single `HttpClient` instance can `handle concurrent requests` and plays well
with features such as **custom** `headers`, `cookies`, and `authentication` schemes.  

- `HttpClient` lets you write and plug in `custom message handlers`. This enables
***mocking*** in **unit tests**, and the `creation of custom pipelines` (for logging, compression, encryption, and so on).  

-`HttpClient` has a **rich and extensible** type system for `headers` and `content`.

to get the `best performance` with ***HttpClient***, you must `reuse the same instance`

In [ ]:
using System.Net.Http;

var client = new HttpClient();

var task1 = client.GetStringAsync ("http://www.linqpad.net");
var task2 = client.GetStringAsync ("http://www.albahari.com");

Console.WriteLine (await task1);
Console.WriteLine (await task2);

***HttpClient*** has a `Timeout` property and a `BaseAddress` property, which **prefixes** a URI to `every request`.

In [ ]:
//most of settings finds in HttpClientHandler
using System.Net.Http;

var handler = new HttpClientHandler { UseProxy = false };
var client = new HttpClient (handler);

### GetAsync and Response Messages

In [ ]:
using System.Net.Http;
using System.IO;

var client = new HttpClient();
HttpResponseMessage response = await client.GetAsync ("http://...");
response.EnsureSuccessStatusCode();// if not success throw exception
string html = await response.Content.ReadAsStringAsync();

//HttpContent has a CopyToAsync method for writing to another stream
using (var fileStream = File.Create ("linqpad.html"))
await response.Content.CopyToAsync (fileStream);

`GetAsync` is one of **four methods** corresponding to HTTP’s four verbs (the others
are `PostAsync`, `PutAsync`, and `DeleteAsync`).

The `GetStringAsync`, `GetByteArrayAsync`, and `GetStreamAsync` methods are convenient ***shortcuts*** for calling the more `general GetAsync method`, which returns a `response message`:

### SendAsync and Request Messages

`SendAsync` is **low-level method** for send http requests. for use this method you first construct an `HttpRequestMessage`

In [ ]:
using System.Net.Http;

var client = new HttpClient();
var request = new HttpRequestMessage (HttpMethod.Get, "http://...");
HttpResponseMessage response = await client.SendAsync (request);
response.EnsureSuccessStatusCode();
string html = await response.Content.ReadAsStringAsync();

`GetAsync`, `PostAsync`, `PutAsync`, and `DeleteAsync` are all **shortcuts** for calling ***SendAsync***

### Uploading Data and HttpContent

After instantiating a `HttpRequestMessage` object, you can upload content by assigning its `Content` property.  
The type for this property is an ***abstract class*** called `HttpContent`.  
  
.NET includes the following concrete **subclasses** for `different kinds of content` (you can also write `your own`):  
- `ByteArrayContent`
- `StringContent`
- `FormUrlEncodedContent` 
- `StreamContent`

In [ ]:
using System.Net.Http;

var client = new HttpClient (new HttpClientHandler { UseProxy = false });
var request = new HttpRequestMessage (
HttpMethod.Post, "http://www.albahari.com/EchoPost.aspx");
request.Content = new StringContent ("This is a test");
HttpResponseMessage response = await client.SendAsync (request);
response.EnsureSuccessStatusCode();
Console.WriteLine (await response.Content.ReadAsStringAsync());

### HttpMessageHandler

most of the properties for customizing requests are defined **not in HttpClient** but in `HttpClientHandler`.  
  
`HttpClientHandler` is **subclass** of `HttpMessageHandler` that is **abstract class**

In [ ]:
using System.Net.Http;
using System.Threading;

public abstract class HttpMessageHandler : IDisposable
{
    protected internal abstract Task<HttpResponseMessage> SendAsync
        (HttpRequestMessage request, CancellationToken cancellationToken);
    public void Dispose(){}
    protected virtual void Dispose (bool disposing){}
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p><code>HttpClientHandler</code> زیرکلاسی از <code>HttpMessageHandler</code> است که برای پیکربندی درخواست‌های HTTP استفاده می‌شود. بیشتر ویژگی‌های سفارشی‌سازی درخواست‌ها در این کلاس تعریف شده‌اند، نه در <code>HttpClient</code>.</p>
<h5>نحوه استفاده در <code>HttpClient</code></h5>
<ul><li>متد <code>SendAsync</code> در کلاس <code>HttpClient</code> در واقع متد <code>SendAsync</code> کلاس <code>HttpMessageHandler</code> را فراخوانی می‌کند. این ساختار اجازه می‌دهد تا کلاس‌های مختلفی را برای مدیریت درخواست‌ها ایجاد و از آنها استفاده کنید.</li><li>کلاس <code>HttpMessageHandler</code> یک نقطه گسترش (Extensibility Point) را فراهم می‌کند که با استفاده از آن می‌توانید رفتار پیش‌فرض <code>HttpClient</code> را تغییر دهید.</li></ul>
</div>

#### Unit testing and mocking

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>در تست‌های واحد، می‌توان از <code>HttpMessageHandler</code> برای ساخت یک mock از درخواست‌های HTTP استفاده کرد. این کار به شما اجازه می‌دهد تا بدون نیاز به انجام درخواست‌های واقعی، رفتار درخواست‌های HTTP را شبیه‌سازی و تست کنید.</p>
</div>

In [ ]:
using System;
using System.Net;
using System.Net.Http;
using System.Threading;
using System.Threading.Tasks;
using Xunit;

public class MockHttpMessageHandler : HttpMessageHandler
{
    private readonly Func<HttpRequestMessage, HttpResponseMessage> _sendAsync;

    public MockHttpMessageHandler(Func<HttpRequestMessage, HttpResponseMessage> sendAsync)
    {
        _sendAsync = sendAsync ?? throw new ArgumentNullException(nameof(sendAsync));
    }

    protected override Task<HttpResponseMessage> SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)
    {
        HttpResponseMessage response = _sendAsync(request);
        return Task.FromResult(response);
    }
}

var mockHandler = new MockHttpMessageHandler(request =>
{
    return new HttpResponseMessage(HttpStatusCode.OK)
    {
        Content = new StringContent("{\"message\": \"Hello, World!\"}", System.Text.Encoding.UTF8, "application/json")
    };
});
// ایجاد یک HttpClient با استفاده از MockHttpMessageHandler
var client = new HttpClient(mockHandler);

 // انجام یک درخواست GET
 var response = await client.GetAsync("https://example.com/api/message");

 // بررسی پاسخ
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();

Assert.Equal("{\"message\": \"Hello, World!\"}", content);

#### Chaining handlers with DelegatingHandler

You can create a message handler that `calls another` (resulting in a chain of handlers) by subclassing `DelegatingHandler`. You can use this to ***implement custom*** `authentication`, `compression`, and `encryption` protocols.

In [ ]:
using System;
using System.Net.Http;
using System.Threading;
using System.Threading.Tasks;

// هندلر برای اضافه کردن هدر سفارشی
public class AddHeaderHandler : DelegatingHandler
{
    protected override async Task<HttpResponseMessage> SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)
    {
        request.Headers.Add("X-Custom-Header", "MyCustomValue");
        return await base.SendAsync(request, cancellationToken);
    }
}

// هندلر برای لاگ کردن درخواست‌ها
public class LogRequestHandler : DelegatingHandler
{
    protected override async Task<HttpResponseMessage> SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)
    {
        Console.WriteLine($"Request: {request}");
        return await base.SendAsync(request, cancellationToken);
    }
}

// هندلر برای لاگ کردن پاسخ‌ها
public class LogResponseHandler : DelegatingHandler
{
    protected override async Task<HttpResponseMessage> SendAsync(HttpRequestMessage request, CancellationToken cancellationToken)
    {
        HttpResponseMessage response = await base.SendAsync(request, cancellationToken);
        Console.WriteLine($"Response: {response}");
        return response;
    }
}

// استفاده از هندلرها به صورت زنجیره‌ای
public class Program
{
    public static async Task Main(string[] args)
    {
        // ایجاد هندلرهای زنجیره‌ای
        var addHeaderHandler = new AddHeaderHandler
        {
            InnerHandler = new LogRequestHandler
            {
                InnerHandler = new LogResponseHandler
                {
                    InnerHandler = new HttpClientHandler()
                }
            }
        };

        // ایجاد HttpClient با استفاده از هندلرهای زنجیره‌ای
        var client = new HttpClient(addHeaderHandler);

        // ارسال درخواست
        var response = await client.GetAsync("https://jsonplaceholder.typicode.com/posts/1");

        // نمایش پاسخ
        string responseBody = await response.Content.ReadAsStringAsync();
        Console.WriteLine($"Response Body: {responseBody}");
    }
}


### Proxies

A `proxy server` is an ***intermediary(واسطه)*** through which ***HTTP requests*** can be `routed`.

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">

<p>پراکسی سرورها نقش واسط بین کاربران و منابع اینترنتی را ایفا می‌کنند و می‌توانند عملکردهای مختلفی داشته باشند. در درخواست‌های HTTP، پراکسی سرورها به طرق مختلف می‌توانند عمل کنند:</p>
<ol><li><p><strong>واسطه‌گری و بازپخش درخواست‌ها (Forward Proxy)</strong>:</p><ul><li>زمانی که یک کاربر درخواست HTTP (مثل درخواست برای مشاهده یک وب‌سایت) را ارسال می‌کند، این درخواست ابتدا به پراکسی سرور ارسال می‌شود.</li><li>پراکسی سرور سپس درخواست را به سرور مقصد (مثلاً سرور وب‌سایت) ارسال می‌کند.</li><li>پاسخ دریافتی از سرور مقصد ابتدا به پراکسی سرور برمی‌گردد و سپس پراکسی سرور پاسخ را به کاربر نهایی ارسال می‌کند.</li></ul></li><li><p><strong>حفظ امنیت و حریم خصوصی</strong>:</p><ul><li>پراکسی سرور می‌تواند اطلاعات هویتی کاربر (مثل آدرس IP) را مخفی کند و به این ترتیب امنیت و حریم خصوصی کاربر را افزایش دهد.</li><li>برخی از پراکسی سرورها می‌توانند درخواست‌ها را رمزنگاری کنند و به این ترتیب از اطلاعات کاربران در برابر حملات میان‌راهی (Man-in-the-middle Attacks) محافظت کنند.</li></ul></li><li><p><strong>کشینگ (Caching)</strong>:</p><ul><li>پراکسی سرورها می‌توانند محتواهای درخواست‌شده را در حافظه خود ذخیره کنند. در صورتی که درخواست مشابهی در آینده دریافت شود، پراکسی سرور می‌تواند پاسخ را از حافظه خود ارسال کند بدون اینکه نیاز به ارسال مجدد درخواست به سرور مقصد باشد.</li><li>این ویژگی می‌تواند سرعت دسترسی به منابع را افزایش دهد و ترافیک شبکه را کاهش دهد.</li></ul></li><li><p><strong>فیلترینگ محتوا</strong>:</p><ul><li>پراکسی سرورها می‌توانند به عنوان فیلتر عمل کنند و دسترسی به برخی وب‌سایت‌ها یا محتواهای خاص را مسدود کنند. این ویژگی معمولاً در محیط‌های سازمانی یا آموزشی برای کنترل دسترسی کاربران به اینترنت استفاده می‌شود.</li></ul></li><li><p><strong>کنترل دسترسی</strong>:</p><ul><li>پراکسی سرورها می‌توانند دسترسی به اینترنت را براساس سیاست‌های مختلف (مثلاً محدود کردن دسترسی براساس زمان، نوع محتوا، یا هویت کاربر) کنترل کنند.</li></ul></li><li><p><strong>بالانس بار (Load Balancing)</strong>:</p><ul><li>برخی پراکسی سرورها می‌توانند درخواست‌ها را به چندین سرور مختلف توزیع کنند تا بار ترافیکی به طور مساوی بین سرورها تقسیم شود و عملکرد کلی شبکه بهبود یابد.</li></ul></li><li><p><strong>فشرده‌سازی داده‌ها</strong>:</p><ul><li>پراکسی سرورها می‌توانند داده‌های HTTP را قبل از ارسال به کاربر فشرده کنند، که این امر می‌تواند باعث کاهش حجم داده‌های انتقالی و افزایش سرعت انتقال شود.</li></ul></li></ol>

</div>

To use a **proxy** with ***HttpClient***, first create an `HttpClientHandler` and assign its `Proxy property` and then feed that into `HttpClient’s constructor`

In [1]:
using System.Net;
using System.Net.Http;

// فرض کنید یک سازمان هستیم که می خواهیم دسترسی به انترنت را 
// فقط از طریق پراکسی محیا کنیم
WebProxy p = new WebProxy ("192.178.10.49", 808);
p.Credentials = new NetworkCredential ("username", "password", "domain");

var handler = new HttpClientHandler { Proxy = p };
var client = new HttpClient (handler);

### Authentication

In [ ]:
using System.Net;
using System.Net.Http;

// You can supply a username and password to an HttpClient as follows:

string username = "myuser";
string password = "mypassword";

var handler = new HttpClientHandler();
handler.Credentials = new NetworkCredential (username, password);

var client = new HttpClient (handler);

This works with ***dialog-based*** authentication protocols, such as `Basic` and `Digest`, and is extensible through the `AuthenticationManager` class.  
  
It also supports `Windows NTLM` and `Kerberos` (if you include a `domain name` when constructing the `NetworkCredential` object).

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p> وقتی شما <code>HttpClient</code> را با اعتبارنامه‌ها تنظیم می‌کنید، فرآیند به این صورت است:</p>

<ol><li><strong>ارسال درخواست اولیه</strong>: <code>HttpClient</code> یک درخواست اولیه به سرور ارسال می‌کند تا منابع مورد نظر را درخواست کند.</li><li><strong>پاسخ سرور</strong>: اگر منبع نیاز به احراز هویت داشته باشد و اعتبارنامه‌های معتبر در درخواست اولیه وجود نداشته باشد، سرور با یک پاسخ 401 Unauthorized پاسخ می‌دهد و در هدرهای پاسخ (مانند WWW-Authenticate) پروتکل‌های احراز هویت مورد قبول خود را اعلام می‌کند.</li><li><strong>مذاکره پروتکل</strong>: <code>HttpClient</code> این هدرها را بررسی می‌کند و پروتکل‌های احراز هویت پشتیبانی شده توسط سرور را شناسایی می‌کند.</li><li><strong>ارسال درخواست احراز هویت شده</strong>: <code>HttpClient</code> یکی از پروتکل‌های پشتیبانی شده را انتخاب می‌کند، اعتبارنامه‌ها را بر اساس آن پروتکل تنظیم می‌کند و سپس یک درخواست جدید با اعتبارنامه‌ها ارسال می‌کند.</li></ol>
</div>

نمونه ای از پاسخ دریافتی از درخواست اولیه

```http
HTTP/1.1 401 Unauthorized
Content-Length: 83
Content-Type: text/html
Server: Microsoft-IIS/6.0
WWW-Authenticate: Negotiate
WWW-Authenticate: NTLM
WWW-Authenticate: Basic realm="exchange.somedomain.com"
X-Powered-By: ASP.NET
Date: Sat, 05 Aug 2006 12:37:23 GMT
```

### CredentialCache

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p><strong>CredentialCache چیست؟</strong></p>
<ul><li><code>CredentialCache</code> یک کلاس در .NET است که برای نگهداری و مدیریت مجموعه‌ای از اعتبارنامه‌ها (<code>NetworkCredential</code> objects) استفاده می‌شود.</li><li>شما می‌توانید چندین اعتبارنامه را در یک <code>CredentialCache</code> ذخیره کنید، هر کدام مرتبط با یک URI و پروتکل خاص.</li></ul>

<p><strong>چرا CredentialCache مفید است؟</strong></p>
<ul><li>اگر نیاز دارید تا برای URI‌های مختلف و یا پروتکل‌های احراز هویت مختلف از اعتبارنامه‌های متفاوتی استفاده کنید، <code>CredentialCache</code> بسیار مفید است.</li><li>به شما این امکان را می‌دهد که به طور خاص تعیین کنید که کدام اعتبارنامه برای کدام URI و کدام پروتکل احراز هویت استفاده شود.</li></ul>
</div>

In [ ]:
using System;
using System.Net;
using System.Net.Http;
using System.Threading.Tasks;

public class Program
{
    public static async Task Main(string[] args)
    {
        // تعریف اعتبارنامه‌ها
        var basicCredential = new NetworkCredential("basicUser", "basicPassword");
        var digestCredential = new NetworkCredential("digestUser", "digestPassword");

        // ایجاد یک CredentialCache و افزودن اعتبارنامه‌ها
        var credentialCache = new CredentialCache();
        credentialCache.Add(new Uri("https://example.com/basic"), "Basic", basicCredential);
        credentialCache.Add(new Uri("https://example.com/digest"), "Digest", digestCredential);

        // ایجاد HttpClientHandler و تنظیم CredentialCache
        var handler = new HttpClientHandler();
        handler.Credentials = credentialCache;

        // ایجاد HttpClient با استفاده از handler
        var client = new HttpClient(handler);

        try
        {
            // ارسال درخواست به سرور با استفاده از پروتکل Basic
            var basicResponse = await client.GetAsync("https://example.com/basic/resource");
            Console.WriteLine($"Basic Auth Response Status: {basicResponse.StatusCode}");

            // ارسال درخواست به سرور با استفاده از پروتکل Digest
            var digestResponse = await client.GetAsync("https://example.com/digest/resource");
            Console.WriteLine($"Digest Auth Response Status: {digestResponse.StatusCode}");
        }
        catch (Exception ex)
        {
            Console.WriteLine($"An error occurred: {ex.Message}");
        }
    }
}


In [ ]:
using System.Net;
using System.Net.Http;

CredentialCache cache = new CredentialCache();
Uri prefix = new Uri ("http://exchange.somedomain.com");
cache.Add (prefix, "Digest", new NetworkCredential ("joe", "passwd"));
cache.Add (prefix, "Negotiate", new NetworkCredential ("joe", "passwd"));

var handler = new HttpClientHandler();
handler.Credentials = cache;
var client = new HttpClient(handler);

//continue

An `authentication protocol` is specified as a string. The **valid values** include:    
  
`Basic`, `Digest`, `NTLM`, `Kerberos`, `Negotiate`

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; border:1px solid rgb(226, 197, 65); padding: 10px">
<h3 style="text-align:center">
<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" fill="none" viewBox="0 0 24 24" class="icon-md" style="color: rgb(118, 208, 235);"><path fill="currentColor" fill-rule="evenodd" d="M12.455 4.105a1 1 0 0 0-.91 0L1.987 8.982 1.077 7.2l9.56-4.877a3 3 0 0 1 2.726 0l9.56 4.877A1.98 1.98 0 0 1 24 9.22V15a1 1 0 1 1-2 0v-3.784l-2.033.995v4.094a3 3 0 0 1-1.578 2.642l-4.967 2.673a3 3 0 0 1-2.844 0l-4.967-2.673a3 3 0 0 1-1.578-2.642v-4.094l-2.927-1.433C-.374 10.053-.39 7.949 1.077 7.2l.91 1.782 9.573 4.689a1 1 0 0 0 .88 0L22 8.989v-.014zM6.033 13.19v3.114a1 1 0 0 0 .526.88l4.967 2.674a1 1 0 0 0 .948 0l4.967-2.673a1 1 0 0 0 .526-.88V13.19l-4.647 2.276a3 3 0 0 1-2.64 0z" clip-rule="evenodd"></path></svg>
Authentication protocols</h3>
<ol><li><p><strong>Basic Authentication</strong>:</p><ul><li><strong>استاندارد</strong>: این پروتکل یک استاندارد عمومی است که در بسیاری از سیستم‌عامل‌ها و سرورها (شامل ویندوز، لینوکس، و دیگر سیستم‌ها) استفاده می‌شود.</li><li><strong>نحوه کار</strong>: نام کاربری و رمز عبور به صورت Base64 رمزگذاری شده و در هدر HTTP ارسال می‌شود.</li><li><strong>امنیت</strong>: به دلیل عدم رمزگذاری قوی، بهتر است تنها در ارتباطات HTTPS استفاده شود.</li></ul></li><li><p><strong>Digest Authentication</strong>:</p><ul><li><strong>استاندارد</strong>: این نیز یک پروتکل استاندارد است که در سرورهای مختلف (ویندوز و لینوکس) پشتیبانی می‌شود.</li><li><strong>نحوه کار</strong>: رمز عبور به صورت هش شده ارسال می‌شود که امنیت بیشتری نسبت به Basic دارد.</li><li><strong>کاربرد</strong>: معمولاً در وب سرورها و سرویس‌های مختلف مورد استفاده قرار می‌گیرد.</li></ul></li><li><p><strong>NTLM</strong>:</p><ul><li><strong>ویژگی‌ها</strong>: این پروتکل بیشتر در محیط‌های ویندوز و شبکه‌های ویندوزی استفاده می‌شود.</li><li><strong>نحوه کار</strong>: یک پروتکل امنیتی مایکروسافت که برای احراز هویت در شبکه‌های مبتنی بر ویندوز توسعه یافته است.</li><li><strong>کاربرد</strong>: اغلب در سرویس‌های داخلی ویندوز مانند IIS و فایل سرورها استفاده می‌شود.</li></ul></li><li><p><strong>Kerberos</strong>:</p><ul><li><strong>استاندارد</strong>: این یک پروتکل استاندارد و باز است که در بسیاری از سیستم‌عامل‌ها شامل ویندوز و لینوکس استفاده می‌شود.</li><li><strong>نحوه کار</strong>: از سیستم تیکتینگ (Ticketing) برای احراز هویت امن استفاده می‌کند.</li><li><strong>کاربرد</strong>: در محیط‌های سازمانی بزرگ برای احراز هویت کاربران به منابع مختلف مورد استفاده قرار می‌گیرد.</li></ul></li><li><p><strong>Negotiate</strong>:</p><ul><li><strong>ویژگی‌ها</strong>: این یک مکانیزم ترکیبی است که می‌تواند از NTLM یا Kerberos استفاده کند.</li><li><strong>نحوه کار</strong>: سیستم به طور خودکار پروتکل مناسب (NTLM یا Kerberos) را بر اساس پشتیبانی سرور و تنظیمات کلاینت انتخاب می‌کند.</li><li><strong>کاربرد</strong>: معمولاً در محیط‌های ویندوزی و شبکه‌هایی که از هر دو پروتکل پشتیبانی می‌کنند استفاده می‌شود.</li></ul></li></ol>
</div>

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; border:1px solid rgb(226, 197, 65); padding: 10px">
<h3 style="text-align:center">
<svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" fill="none" viewBox="0 0 24 24" class="icon-md" style="color: rgb(118, 208, 235);"><path fill="currentColor" fill-rule="evenodd" d="M12.455 4.105a1 1 0 0 0-.91 0L1.987 8.982 1.077 7.2l9.56-4.877a3 3 0 0 1 2.726 0l9.56 4.877A1.98 1.98 0 0 1 24 9.22V15a1 1 0 1 1-2 0v-3.784l-2.033.995v4.094a3 3 0 0 1-1.578 2.642l-4.967 2.673a3 3 0 0 1-2.844 0l-4.967-2.673a3 3 0 0 1-1.578-2.642v-4.094l-2.927-1.433C-.374 10.053-.39 7.949 1.077 7.2l.91 1.782 9.573 4.689a1 1 0 0 0 .88 0L22 8.989v-.014zM6.033 13.19v3.114a1 1 0 0 0 .526.88l4.967 2.674a1 1 0 0 0 .948 0l4.967-2.673a1 1 0 0 0 .526-.88V13.19l-4.647 2.276a3 3 0 0 1-2.64 0z" clip-rule="evenodd"></path></svg>
Other Authentication protocols</h3>
<h5>OAuth و OAuth2</h5>
<ul><li><strong>ویژگی‌ها</strong>: OAuth یک استاندارد باز برای احراز هویت و اجازه دسترسی است. OAuth2 نسخه جدیدتر و گسترده‌تر آن است.</li><li><strong>نحوه کار</strong>: به کاربران اجازه می‌دهد تا بدون اشتراک‌گذاری اعتبارنامه‌های خود با سرویس‌های شخص ثالث، به منابع خود دسترسی بدهند.</li><li><strong>کاربرد</strong>: به طور گسترده در برنامه‌های وب و موبایل برای احراز هویت و اجازه دسترسی به APIها استفاده می‌شود. (مانند Google, Facebook, GitHub)</li></ul>

<h5>OpenID و OpenID Connect</h5>
<ul><li><strong>ویژگی‌ها</strong>: OpenID یک پروتکل برای احراز هویت کاربر است. OpenID Connect یک لایه روی OAuth2 است که احراز هویت را به OAuth2 اضافه می‌کند.</li><li><strong>نحوه کار</strong>: کاربران می‌توانند از یک شناسه واحد برای ورود به چندین سایت استفاده کنند.</li><li><strong>کاربرد</strong>: احراز هویت متمرکز برای دسترسی به وبسایت‌های مختلف با استفاده از یک حساب کاربری (مانند Google Login).</li></ul>
<h5>JWT (JSON Web Token)</h5>
<ul><li><strong>ویژگی‌ها</strong>: JWT یک استاندارد باز برای ایجاد توکن‌های امن و قابل تأیید است که حاوی اطلاعات کاربر است.</li><li><strong>نحوه کار</strong>: توکن‌ها به صورت JSON رمزگذاری شده و بین کلاینت و سرور رد و بدل می‌شوند.</li><li><strong>کاربرد</strong>: احراز هویت و اجازه دسترسی در برنامه‌های وب و موبایل.</li></ul>

<h5>SAML (Security Assertion Markup Language)</h5>
<h5>LDAP (Lightweight Directory Access Protocol)</h5>
<h5>RADIUS (Remote Authentication Dial-In User Service)</h5>
<h5>TACACS+ (Terminal Access Controller Access-Control System Plus)</h5>
<h5>Mutual TLS (mTLS)</h5>
</div>

#### Authenticating via headers

Another way to ***authenticate*** is to set the `authentication header` directly:

In [ ]:
using System.Net;
using System.Net.Http.Headers;

var client = new HttpClient();
client.DefaultRequestHeaders.Authorization =
    new AuthenticationHeaderValue ("Basic",
    Convert.ToBase64String (Encoding.UTF8.GetBytes ("username:password")));

This strategy also works with `custom authentication` systems such as `OAuth` and `JWT`.

### Headers

***HttpClient*** lets you add custom `HTTP headers` to a request, as well as ***enumerate*** the `headers in a response`.  
  
A ***header*** is simply a `key/value pair` containing metadata, such as the message content type or server software.

In [ ]:
// two way for add Headers

var client = new HttpClient();
client.DefaultRequestHeaders.Add("Global-Header", "GlobalHeaderValue");
// add header for all reqeust that send with this HttpClient

var request1 = new HttpRequestMessage(HttpMethod.Post, "https://www.example1.com");
request1.Headers.Add("Request-Header-1", "HeaderValue1");

var response1 = await client.SendAsync(request1);
Console.WriteLine(await response1.Content.ReadAsStringAsync());

var request2 = new HttpRequestMessage(HttpMethod.Get, "https://www.example2.com");
request2.Headers.Add("Request-Header-2", "HeaderValue2");

var response2 = await client.SendAsync(request2);
Console.WriteLine(await response2.Content.ReadAsStringAsync());

### Query Strings

A ***query string*** is simply a string appended to a URI with a `question mark`

In [ ]:
string requestURI = "http://www.google.com/search?q=HttpClient&hl=fr";

If there’s a possibility of ***your query*** including `symbols` or `spaces`, you can use Uri’s `EscapeDataString` method to create a ***legal URI***

In [2]:
string search = Uri.EscapeDataString ("(HttpClient or HttpRequestMessage)");
string language = Uri.EscapeDataString ("fr");
string requestURI = "http://www.google.com/search?q=" + search +
                    "&hl=" + language;
Console.WriteLine(requestURI);

http://www.google.com/search?q=%28HttpClient%20or%20HttpRequestMessage%29&hl=fr


### Uploading Form Data

To upload `HTML form data`, create and populate the `FormUrlEncodedContent` object.

In [ ]:
string uri = "http://www.albahari.com/EchoPost.aspx";
var client = new HttpClient();
var dict = new Dictionary<string,string>
{
    { "Name", "Joe Albahari" },
    { "Company", "O'Reilly" }
};
var values = new FormUrlEncodedContent (dict);
var response = await client.PostAsync (uri, values);
response.EnsureSuccessStatusCode();
Console.WriteLine (await response.Content.ReadAsStringAsync());

### Cookies

A **cookie** is a `name/value` string pair that an **HTTP server** sends to a `client` in a ***response header***.  
  
A `web browser client` typically ***remembers cookies*** and `replays them` to the server in ***each subsequent request*** (to the `same address`) until their `expiry`.

***By default***, `HttpClient` **ignores** any cookies received from the server.  
  
***To accept cookies***, create a `CookieContainer` object and assign it an `HttpClientHandler`

In [ ]:
var cc = new CookieContainer();
var handler = new HttpClientHandler();
handler.CookieContainer = cc;
var client = new HttpClient (handler);